## Face Valiation


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
! pip install tmdbsimple
! pip install mtcnn 
!pip install --upgrade fastai
from fastai.vision.all import *

     |████████████████████████████████| 2.3MB 8.2MB/s 
     |████████████████████████████████| 194kB 6.5MB/s 
     |████████████████████████████████| 61kB 10.2MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


# By Races

## Building The Directories

In [ ]:
races = {
'black':['Chadwick Boseman','Will Smith','Morgan Freeman','Donald Glover','Forest Whitaker','Michael Ealy','Halle Berry','Angela Bassett','Viola Davis','Cicely Tyson'],
'asian':['Awkwafina','Lana Condor','Aubrey Anderson-Emmons','Lyrica Okano','Claudia Kim','Ki Hong Lee','Manny Jacinto','Arden Cho','Ian Chen','Henry Golding'],
'indian':['Amitabh Bachchan','Priyanka Chopra','Irrfan Khan','Akshay Kumar','Deepika Padukone','Anil Kapoor','Kamal Haasan','Vidya Balan','Kareena Kapoor','Katrina Kaif'],
'hispanic':['Jennifer Lopez','John Leguizamo','Cheech Marin','Danny Trejo','Sofía Vergara','Penélope Cruz','George Lopez','Benjamin Bratt','Guillermo Navarro','Aubrey Plaza'],
'white':['James Mcavoy','Chris Pratt','Chris Hemsworth','Chris Evans','Sebastian Stan','Angelina Jolie','Scarlett Johansson','Christina Hendricks','Amanda Seyfried','Natalie Portman']
}

actors = ['Chadwick Boseman','Will Smith','Morgan Freeman','Donald Glover','Forest Whitaker','Michael Ealy','Halle Berry','Angela Bassett','Viola Davis','Cicely Tyson','Awkwafina','Lana Condor','Aubrey Anderson-Emmons','Lyrica Okano','Claudia Kim','Ki Hong Lee','Manny Jacinto','Arden Cho','Ian Chen','Henry Golding','Amitabh Bachchan','Priyanka Chopra','Irrfan Khan','Akshay Kumar','Deepika Padukone','Anil Kapoor','Kamal Haasan','Vidya Balan','Kareena Kapoor','Katrina Kaif','Jennifer Lopez','John Leguizamo','Cheech Marin','Danny Trejo','Sofía Vergara','Penélope Cruz','George Lopez','Benjamin Bratt','Guillermo Navarro','Aubrey Plaza','James Mcavoy','Chris Pratt','Chris Hemsworth','Chris Evans','Sebastian Stan','Angelina Jolie','Scarlett Johansson','Christina Hendricks','Amanda Seyfried','Natalie Portman']
actors_ids = [172069,2888,192,119589,2178,8177,4587,9780,19492,18249,1625558,1452046,1224228,1684139,1401531,1310760,1368795,144279,1397359,1807132,35780,77234,76793,35070,53975,72118,93193,35068,37233,81869,16866,5723,11159,11160,63522,955,41798,4589,3113,119592,5530,73457,74568,16828,60898,11701,1245,110014,71070,524]

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import tmdbsimple as tmdb
from pathlib import Path
import pandas as pd

Path("validation").mkdir(exist_ok=True)
Path("validation/actors").mkdir(exist_ok=True)
Path("validation/races").mkdir(exist_ok=True)

base_dir = Path('validation')
actors_dir = base_dir / 'actors'
races_dir = base_dir / 'races'

tmdb.API_KEY = 
base_url = 'https://image.tmdb.org/t/p/w780'
search = tmdb.Search()

def get_ids(actors_names):
  actors_ids = []

  for actor_name in actors_names:
    search.person(query=actor_name)
    id = search.results[0]['id']
    actors_ids.append(id)

  return actors_ids

def save_actor_images(actor_id, limit):
  people = tmdb.People(actor_id)
  urls = list(map(lambda x: x['file_path'], people.images()['profiles']))

  for i, url in enumerate(urls):
    if i==limit:
      break

    Path(f'{actors_dir}/{people.info()["name"]}').mkdir(exist_ok=True)

    response = requests.get(f'{base_url}{url}')
    img = Image.open(BytesIO(response.content))
    img.save(f'{actors_dir}/{people.info()["name"]}/{people.info()["name"]}-{i}.jpg')

def save_actors_by_race(actors_names, race):
  actors_ids = get_ids(actors_names)
  for actor_id in actors_ids:
    people = tmdb.People(actor_id)
    url = people.images()['profiles'][0]['file_path']

    Path(f'{races_dir}/{race}').mkdir(exist_ok=True)

    response = requests.get(f'{base_url}{url}')
    img = Image.open(BytesIO(response.content))
    img.save(f'{races_dir}/{race}/{people.info()["name"]}.jpg')

for id in actors_ids:
  save_actor_images(id, 3)

for race, names in races.items():
  save_actors_by_race(names, race)

## Cropping

In [ ]:
Path('cropped').mkdir(exist_ok=True)
!cp -r validation/* cropped/

In [ ]:
from mtcnn import MTCNN
import cv2


detector = MTCNN()

from tqdm import tqdm_notebook as tqdm
cropped_path= Path('cropped')

for img in tqdm(cropped_path.rglob("*.jpg"), desc="Images", total=len(list(cropped_path.rglob("*.jpg")))):
  try: 
    im = Image.open(img).convert("RGB") 
    for face in detector.detect_faces(np.asarray(im)):
      box = face["box"]
      im1 = im.crop(box=(box[0], box[1], box[0]+box[2],box[1]+box[3] )) 
      im1.save(img)
      break
  except IsADirectoryError:
    pass

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
cropped_dir = Path(f'cropped')
cropped_actors_dir = cropped_dir / 'actors'
cropped_races_dir = cropped_dir / 'races'

df_val = pd.DataFrame({"img": list(cropped_races_dir.rglob("*.jpg")), "img_raw": list(races_dir.rglob("*.jpg"))})
df_val["label"] = df_val["img"].apply(lambda x: x.parents[0].name)
df_val["name"] = df_val["img"].apply(lambda x: x.stem)

## Our Models

In [ ]:
model_acc = {}

In [ ]:
models_dir = Path(root_dir)/"models"
models_paths = list(models_dir.glob("*.pkl"))

for m_path in tqdm(models_paths[6:7]):
  print(m_path)
  learner = load_learner(m_path)
  res = [learner.predict(line)[0] for line in df_val["img"]]

  df_val["Ours"] = res
  df_val["Ours"] = df_val["Ours"].replace("latino","hispanic")
  df_val["Ours"] = df_val["Ours"].replace("african","black")
  acc = sum(df_val["Ours"] == df_val["label"]) / len(df_val)
  model_acc[m_path.stem] = acc

model_acc

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


{'ethni_0825f1_v6': 0.52,
 'ethni_085b_96unfrezee_V6': 0.58,
 'ethni_085b_unfrezee_V6': 0.58,
 'ethni_086f1': 0.54}

In [ ]:
models_paths[1:2]
models_paths[2:3]
learner = load_learner()

AttributeError: ignored

## Fair Face Model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = Path('dataset/')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/dchen236/FairFace.git

Cloning into 'FairFace'...
remote: Enumerating objects: 202, done.
remote: Total 202 (delta 0), reused 0 (delta 0), pack-reused 202
Receiving objects: 100% (202/202), 14.23 MiB | 38.84 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [ ]:
!cp /content/gdrive/My\ Drive/Ethnicity/Fair/fair_face_models.zip FairFace/fair_face_models.zip

In [ ]:
!unzip FairFace/fair_face_models.zip -d FairFace

Archive:  FairFace/fair_face_models.zip
   creating: FairFace/fair_face_models/
  inflating: FairFace/fair_face_models/fairface_alldata_4race_20191111.pt  
  inflating: FairFace/fair_face_models/res34_fair_align_multi_7_20190809.pt  


In [ ]:
!mv FairFace/fair_face_models/res34_fair_align_multi_7_20190809.pt FairFace/fair_face_models/fairface_alldata_20191111.pt

In [ ]:
df_val["img_path"] = df_val["img_raw"].apply(lambda x: str(x.absolute()))
df_val[["img_path"]].to_csv("FairFace/val.csv",index=False)
# df_val["img_face_path"] = df_val["img"].apply(lambda x: str(x.absolute()))

In [ ]:
%cd FairFace

/content/FairFace


In [ ]:
!python predict.py --csv "val.csv"

using CUDA?: True
---0/50---
detected faces are saved at  detected_faces
Predicting... 0/55
saved results at  test_outputs.csv


In [ ]:
%cd ..

/content


In [ ]:
df_fair = pd.read_csv("FairFace/test_outputs.csv")
df_fair["name"] =  df_fair["face_name_align"].apply(lambda x: x.split("/")[-1].split("_face")[0]) 

In [ ]:
df_val["label"] = df_val["label"].replace("asian","Asian")
df_val["label"] = df_val["label"].replace("black","Black")
df_val["label"] = df_val["label"].replace("white","White")
df_val["label"] = df_val["label"].replace("hispanic","White")
df_val["label"] = df_val["label"].replace("indian","Indian")

df_merged = df_fair.merge(df_val)[["race","race4","label"]]

sum(df_merged["race4"] == df_merged["label"]) / len(df_val)

0.9

# By Actors

## Fair Face Model

In [ ]:
actors_dir
[actors_dir / a for a in actors]

[Path('validation/actors/Chadwick Boseman'),
 Path('validation/actors/Will Smith'),
 Path('validation/actors/Morgan Freeman'),
 Path('validation/actors/Donald Glover'),
 Path('validation/actors/Forest Whitaker'),
 Path('validation/actors/Michael Ealy'),
 Path('validation/actors/Halle Berry'),
 Path('validation/actors/Angela Bassett'),
 Path('validation/actors/Viola Davis'),
 Path('validation/actors/Cicely Tyson'),
 Path('validation/actors/Awkwafina'),
 Path('validation/actors/Lana Condor'),
 Path('validation/actors/Aubrey Anderson-Emmons'),
 Path('validation/actors/Lyrica Okano'),
 Path('validation/actors/Claudia Kim'),
 Path('validation/actors/Ki Hong Lee'),
 Path('validation/actors/Manny Jacinto'),
 Path('validation/actors/Arden Cho'),
 Path('validation/actors/Ian Chen'),
 Path('validation/actors/Henry Golding'),
 Path('validation/actors/Amitabh Bachchan'),
 Path('validation/actors/Priyanka Chopra'),
 Path('validation/actors/Irrfan Khan'),
 Path('validation/actors/Akshay Kumar'),
 Pa

In [ ]:
df_actors = pd.DataFrame({"img_path": list(actors_dir.rglob("*.jpg"))})
df_actors["img_path"] = df_actors["img_path"].apply(lambda x: str(x.absolute()))
df_actors["name"] = df_actors["img_path"].apply(lambda p: p.partition("actors/")[2].partition("/")[0])
for i, row in df_val.iterrows():
  df_actors.loc[df_actors["name"] == row["name"],"label"] = row["label"]

df_actors

,img_path,name,label
0,/content/validation/actors/Katrina Kaif/Katrina Kaif-1.jpg,Katrina Kaif,Indian
1,/content/validation/actors/Katrina Kaif/Katrina Kaif-0.jpg,Katrina Kaif,Indian
2,/content/validation/actors/Katrina Kaif/Katrina Kaif-2.jpg,Katrina Kaif,Indian
3,/content/validation/actors/Chris Hemsworth/Chris Hemsworth-1.jpg,Chris Hemsworth,White
4,/content/validation/actors/Chris Hemsworth/Chris Hemsworth-2.jpg,Chris Hemsworth,White
...,...,...,...
118,/content/validation/actors/Henry Golding/Henry Golding-0.jpg,Henry Golding,Asian
119,/content/validation/actors/Henry Golding/Henry Golding-2.jpg,Henry Golding,Asian
120,/content/validation/actors/Henry Golding/Henry Golding-1.jpg,Henry Golding,Asian
121,/content/validation/actors/George Lopez/George Lopez-0.jpg,George Lopez,White


In [ ]:
df_actors[["img_path"]].to_csv("FairFace/actors_val.csv",index=False)

In [ ]:
!rm -rf FairFace/detected_faces/
%cd FairFace
!python predict.py --csv "actors_val.csv"
%cd ..

/content/FairFace
using CUDA?: True
---0/123---
detected faces are saved at  detected_faces
Predicting... 0/124
saved results at  test_outputs.csv
/content


In [ ]:
df_fair_actors = pd.read_csv("FairFace/test_outputs.csv")
df_fair_actors["name"] =  df_fair_actors["face_name_align"].apply(lambda x: x.split("/")[-1].split("_face")[0].split("-")[0]) 
# df_fair_actors[df_fair_actors["name"] == 'John Leguizamo']
group= df_fair_actors[df_fair_actors["name"] == 'John Leguizamo']
# def grouping(g):
  
#   return g.value_counts("race4")
# # df_fair_actors.groupby("name").apply(grouping)
# grouping(group).argmax()

In [ ]:
def make_array(text):
    return np.fromstring(text,sep=' ')



In [ ]:
 np.fromstring(df_fair_actors["race_scores_fair"][0], sep=" ")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  """Entry point for launching an IPython kernel.


array([], dtype=float64)

In [ ]:
df_fair_actors["race_scores_fair"] = df_fair_actors["race_scores_fair"].str.strip("[")
df_fair_actors["race_scores_fair"] = df_fair_actors["race_scores_fair"].str.strip("]")
df_fair_actors["race_scores_fair"] = df_fair_actors["race_scores_fair"].str.replace("\n", "")
df_fair_actors["race_scores_fair"] = df_fair_actors["race_scores_fair"].apply(make_array)

In [ ]:
df_fair_actors["race_scores_fair_4"][0]

'[1.2332422e-04 9.9935174e-01 5.5991706e-05 4.6895538e-04]'

In [ ]:
df_fair_actors["race_scores_fair_4"].apply(make_array)

0      [0.00012332422, 0.99935174, 5.5991706e-05, 0.00046895538]
1         [0.99296612, 3.7447382e-05, 0.001092559, 0.0059038997]
2        [0.0052239671, 5.8794071e-06, 0.99259251, 0.0021776273]
3          [0.40811849, 0.00050312985, 0.0018550161, 0.58952332]
4               [0.84233147, 0.08022348, 0.00153142, 0.07591364]
                                 ...                            
119     [0.99745798, 1.1741641e-05, 1.7528706e-05, 0.0025127043]
120        [0.80446827, 0.0033674056, 0.00041935447, 0.19174498]
121      [0.00028028144, 0.99258035, 0.0036407569, 0.0034986387]
122             [0.98209435, 0.00121475, 0.01133319, 0.00535764]
123    [0.99883103, 3.0531228e-05, 0.00025408514, 0.00088434666]
Name: race_scores_fair_4, Length: 124, dtype: object

In [ ]:
df_fair_actors["race_scores_fair_4"] = df_fair_actors["race_scores_fair_4"].str.strip("[")
df_fair_actors["race_scores_fair_4"] = df_fair_actors["race_scores_fair_4"].str.strip("]")
df_fair_actors["race_scores_fair_4"] = df_fair_actors["race_scores_fair_4"].str.replace("\n", "")
df_fair_actors["race_scores_fair_4"] = df_fair_actors["race_scores_fair_4"].apply(make_array)

In [ ]:
res = []
for n, line in df_fair_actors.groupby("name"):
  res.append((n, races[np.stack(line["race_scores_fair_4"].values).mean(axis=0).argmax()]))
  

In [ ]:
df_fair_actors

,face_name_align,race,race4,gender,age,race_scores_fair,race_scores_fair_4,gender_scores_fair,age_scores_fair,name
0,detected_faces/Viola Davis-1_face0.jpg,Black,Black,Female,30-39,"[1.5005918e-05, 0.99840003, 0.0006869362, 2.85928e-05, 0.00035486693, 0.00051262492, 1.8353982e-06]","[0.00012332422, 0.99935174, 5.5991706e-05, 0.00046895538]",[3.9583317e-04 9.9960417e-01],[1.1529766e-07 6.5544809e-06 3.2189407e-03 2.2442445e-01 6.2065440e-01\n 1.4239049e-01 9.1252709e-03 1.7564757e-04 4.0849932e-06],Viola Davis
1,detected_faces/Angelina Jolie-1_face0.jpg,White,White,Female,30-39,"[0.94232213, 3.5342923e-06, 0.0067687379, 4.2068594e-05, 3.1999098e-05, 0.00022261257, 0.050608966]","[0.99296612, 3.7447382e-05, 0.001092559, 0.0059038997]",[8.1347770e-07 9.9999917e-01],[9.5151993e-09 1.5337221e-06 1.2331266e-03 2.3999824e-01 6.1994410e-01\n 1.3492398e-01 3.8483606e-03 5.0011033e-05 6.2733187e-07],Angelina Jolie
2,detected_faces/Claudia Kim-0_face0.jpg,East Asian,Asian,Female,20-29,"[0.0041960389, 3.305265e-06, 0.0056678024, 0.76874357, 0.22050139, 0.00075009512, 0.00013788353]","[0.0052239671, 5.8794071e-06, 0.99259251, 0.0021776273]",[1.5864788e-06 9.9999845e-01],[2.6334037e-09 1.8922587e-05 6.6879764e-02 8.7879109e-01 5.4132152e-02\n 1.7775259e-04 3.0603212e-07 6.2321988e-09 4.6670379e-11],Claudia Kim
3,detected_faces/Katrina Kaif-2_face0.jpg,Middle Eastern,Indian,Female,20-29,"[0.013114717, 1.994434e-05, 0.024308346, 0.00012815814, 6.078044e-05, 0.13689351, 0.82547456]","[0.40811849, 0.00050312985, 0.0018550161, 0.58952332]",[1.946678e-05 9.999805e-01],[4.9788702e-09 4.4012482e-05 5.2275483e-02 9.0891123e-01 3.8470667e-02\n 2.9762252e-04 1.0330988e-06 1.3739409e-08 2.1042143e-10],Katrina Kaif
4,detected_faces/Jennifer Lopez-1_face0.jpg,Latino_Hispanic,White,Female,20-29,"[0.3261635, 0.0080899969, 0.48276311, 0.00047674504, 0.00067009189, 0.017187383, 0.16464913]","[0.84233147, 0.08022348, 0.00153142, 0.07591364]",[9.2069672e-07 9.9999905e-01],[2.9853867e-10 5.7983875e-06 4.4957239e-02 8.2373488e-01 1.3085344e-01\n 4.4833828e-04 3.2250745e-07 1.9224908e-09 1.9832086e-11],Jennifer Lopez
...,...,...,...,...,...,...,...,...,...,...
119,detected_faces/Natalie Portman-2_face0.jpg,White,White,Female,20-29,"[0.99448234, 9.4419539e-07, 0.0028932542, 1.3004833e-06, 2.2752386e-06, 7.404849e-05, 0.0025458036]","[0.99745798, 1.1741641e-05, 1.7528706e-05, 0.0025127043]",[3.2716875e-06 9.9999666e-01],[1.7046419e-09 1.0830123e-05 2.7543588e-02 7.9106748e-01 1.8010315e-01\n 1.2731085e-03 1.7942209e-06 9.6572883e-09 1.9230553e-10],Natalie Portman
120,detected_faces/Kamal Haasan-2_face0.jpg,Middle Eastern,White,Male,30-39,"[0.17668062, 0.0014569843, 0.036903709, 0.0001834705, 0.0003449845, 0.02403173, 0.76039845]","[0.80446827, 0.0033674056, 0.00041935447, 0.19174498]",[0.9978204 0.0021797],[4.1017915e-06 2.7387670e-05 3.1400399e-04 3.2411549e-02 5.6658798e-01\n 3.8641918e-01 1.4067625e-02 1.6265486e-04 5.4752104e-06],Kamal Haasan
121,detected_faces/Cicely Tyson-0_face0.jpg,Black,Black,Female,40-49,"[0.00014839679, 0.98936987, 0.006893991, 0.00014455509, 0.00043224343, 0.002922701, 8.8181361e-05]","[0.00028028144, 0.99258035, 0.0036407569, 0.0034986387]",[1.9818723e-04 9.9980181e-01],[7.1278043e-07 8.1300177e-06 1.3012382e-04 3.6881329e-03 1.2306122e-01\n 5.2300119e-01 2.9875571e-01 4.9627963e-02 1.7268320e-03],Cicely Tyson
122,detected_faces/Sofía Vergara-0_face0.jpg,White,White,Female,20-29,"[0.88876021, 0.00036526992, 0.10301862, 0.0021632402, 0.0021217964, 0.00076174428, 0.0028090223]","[0.98209435, 0.00121475, 0.01133319, 0.00535764]",[1.4732128e-06 9.9999851e-01],[9.6354995e-09 2.5665344e-05 4.2832222e-02 7.7841109e-01 1.7711672e-01\n 1.6095194e-03 4.7371950e-06 3.0058416e-08 3.4526493e-10],Sofía Vergara


In [ ]:
name_race = df_actors[["name","label"]].drop_duplicates()

In [ ]:
name_race = name_race.merge(df_fair_actors, on="name")

In [ ]:
sum(name_race["race4"]==name_race["label"])

104

In [ ]:
104/len(name_race)

0.8455284552845529

In [ ]:
44/49

0.8979591836734694

In [ ]:
name_race = df_actors[["name","label"]].drop_duplicates()

In [ ]:
pred_avg = pd.DataFrame(res, columns=["Actor","Pred_Label"])

In [ ]:
name_race = name_race.merge(pred_avg, left_on="name", right_on="Actor")

In [ ]:
sum(name_race["label"]==name_race["Pred_Label"])

44

In [ ]:
races = ["White" ,"Black", "Asian" ,"Indian"]

In [ ]:
line

,face_name_align,race,race4,gender,age,race_scores_fair,race_scores_fair_4,gender_scores_fair,age_scores_fair,name
18,detected_faces/Akshay Kumar-0_face0.jpg,Middle Eastern,White,Male,40-49,"[0.29083887, 0.00068838813, 0.16762526, 0.00022458847, 0.00090689614, 0.10402437, 0.43569162]","[0.60454249, 0.00050376414, 0.0018093099, 0.3931444]",[9.9963516e-01 3.6483916e-04],[3.3712990e-06 1.5460555e-05 3.7065023e-04 1.7930385e-02 4.4393662e-01\n 5.1113671e-01 2.6472047e-02 1.3235197e-04 2.4199885e-06],Akshay Kumar
25,detected_faces/Akshay Kumar-2_face0.jpg,White,White,Male,40-49,"[0.27569026, 0.0011593, 0.26068318, 0.00036179, 0.00145879, 0.19555828, 0.26508847]","[0.5320408, 0.00078573, 0.00286712, 0.46430635]",[9.996647e-01 3.352756e-04],[3.9371857e-06 2.1215970e-05 4.5332735e-04 2.0754470e-02 4.5979285e-01\n 4.9296132e-01 2.5888428e-02 1.2239178e-04 2.0850041e-06],Akshay Kumar
87,detected_faces/Akshay Kumar-1_face0.jpg,Middle Eastern,White,Male,40-49,"[0.16456974, 0.00070177554, 0.15480457, 0.00020752005, 0.0010028782, 0.14877318, 0.52994037]","[0.52213955, 0.00066477, 0.00245686, 0.47473887]",[9.995116e-01 4.884003e-04],[5.2415917e-06 3.1099938e-05 5.8093108e-04 2.2633120e-02 4.5965272e-01\n 4.9116763e-01 2.5788864e-02 1.3755726e-04 2.8600330e-06],Akshay Kumar


# old...

## Getting the data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = Path('dataset/')

Mounted at /content/gdrive


In [ ]:
gdrive_path = "/content/gdrive/My\ Drive/"
base_dir = Path('validation/')
path_img = base_dir/ "images"
val_face_path= base_dir/ "Val"
val_raw_path= base_dir/ "Val_Raw"
# fair_path = Path(root_dir)/"Ethnicity/Fair"
models_path = Path(root_dir)/"models/"

In [ ]:
!unzip /content/gdrive/My\ Drive/Ethnicity/Fair/fair_face_models.zip

Archive:  /content/gdrive/My Drive/Ethnicity/Fair/fair_face_models.zip
   creating: fair_face_models/
  inflating: fair_face_models/fairface_alldata_4race_20191111.pt  
  inflating: fair_face_models/res34_fair_align_multi_7_20190809.pt  


In [ ]:
!git clone https://github.com/dchen236/FairFace.git

Cloning into 'FairFace'...
remote: Enumerating objects: 202, done.
remote: Total 202 (delta 0), reused 0 (delta 0), pack-reused 202
Receiving objects: 100% (202/202), 14.23 MiB | 35.19 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [ ]:
Path('FairFace').mkdir(exist_ok=True)
!cp /content/gdrive/My\ Drive/models/Ethnicity/Fair/fair_face_models.zip FairFace/fair_face_models.zip

In [ ]:
!mkdir dataset
!cp -r /content/gdrive/My\ Drive/Ethnicity/Val {str(base_dir)+"/Val"}
!cp -r /content/gdrive/My\ Drive/Ethnicity/Val {str(base_dir)+"/Val_Raw"}


In [ ]:
!pip install mtcnn 

     |████████████████████████████████| 2.3MB 8.4MB/s 


In [ ]:
from mtcnn import MTCNN
import cv2


detector = MTCNN()


In [ ]:
from tqdm import tqdm_notebook as tqdm
val_path= base_dir/ "Val"

for img in tqdm(val_path.rglob("*"), desc="Images"):
  try: 
    im = Image.open(img) 
    for face in detector.detect_faces(np.asarray(im)):
      box = face["box"]
      im1 = im.crop(box=(box[0], box[1], box[0]+box[2],box[1]+box[3] )) 
      im1.save(img)
      break
  except IsADirectoryError:
    pass

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
from PIL import ImageOps
import albumentations as A

class AlbumentationsTransform(DisplayedTransform):
    split_idx,order=0,2
    def __init__(self, train_aug): store_attr()
    
    def encodes(self, img: PILImage):
      if random.random() > 0.5:
        return ImageOps.grayscale(img).convert("RGB")
      else:
        return img
        # aug_img = self.train_aug(image=np.array(img))['image']
        # return PILImage.create(aug_img)
gray = AlbumentationsTransform(A.ToGray(p=0.5))

## Classification

In [ ]:
list(base_dir.rglob("*.jpg"))

In [ ]:
df_val = pd.DataFrame({"img": list(base_dir.rglob("*.jpg")), "img_raw": list(base_dir.rglob("*.jpg"))})
df_val["label"] = df_val["img"].apply(lambda x: x.parents[0].name)

In [ ]:
df_val

In [ ]:
df_val["name"] = df_val["img"].apply(lambda x: x.stem)

In [ ]:
df_val["name"] =  df_val["name"].apply(lambda x: x.split(".")[0]) 

In [ ]:
# df_val["label"] = df_val["label"].replace("asian","Asian")
df_val["label"] = df_val["label"].replace("latino","Latino_Hispanic")
df_val["label"] = df_val["label"].replace("white","White")
df_val["label"] = df_val["label"].replace("african","Black")
df_val["label"] = df_val["label"].replace("asian","East Asian")
df_val["label"] = df_val["label"].replace("indian","Indian")

In [ ]:
df_val["label2"]  = df_val["label"] 
df_val["label2"] = df_val["label2"].replace("East Asian","Asian")
df_val["label2"] = df_val["label2"].replace("Latino_Hispanic","White")

## Dima

In [ ]:
models = ['ethni_0825f1_v6','ethni_083f_unfrezee_V9','ethni_0858f_unfrezee_V9','ethni_085b_96unfrezee_V6','ethni_085b_unfrezee_V6','ethni_086f1']
learner = load_learner(models_path/"ethni_0858f_unfrezee_V9.pkl")

In [ ]:
# learn.export(Path(root_dir)/"Ethnicity/LWF/ethni_new_v9_08.pkl")
# learn.export(Path(root_dir)/"Ethnicity/LWF/ethni_new_v8_082.pkl")
learner = load_learner(models_path/"ethni_0858f_unfrezee_V9.pkl")

In [ ]:
res = [learner.predict(line)[0] for line in df_val["img_face_path"]]

In [ ]:
# [learner.predict(line)[2][:4] for line in df_val["img_face_path"]]

In [ ]:
df_val["Ours"] = res
df_val["Ours"] = df_val["Ours"].replace("asian","Asian")
df_val["Ours"] = df_val["Ours"].replace("latino","White")
df_val["Ours"] = df_val["Ours"].replace("hispanic","White")
df_val["Ours"] = df_val["Ours"].replace("black","Black")
df_val["Ours"] = df_val["Ours"].replace("white","White")
df_val["Ours"] = df_val["Ours"].replace("african","Black")
df_val["Ours"] = df_val["Ours"].replace("asian","Asian")
df_val["Ours"] = df_val["Ours"].replace("indian","Indian")

In [ ]:
sum(df_val["Ours"] == df_val["label2"]) / len(df_val)

0.86

In [ ]:
sum(df2["race4"] == df2["label2"]) / len(df_val)

0.8

In [ ]:
sum(df2["race"] == df2["label"]) / len(df_val)

0.7

In [ ]:
learner.predict(list(df_val["img_face_path"]))

AssertionError: ignored

In [ ]:
model_acc = {}
for m_path in tqdm(list(models_path.glob("*.pkl"))):
  learner = load_learner(m_path)
  res = [learner.predict(line)[0] for line in df_val["img_face_path"]]

  df_val["Ours"] = res
  df_val["Ours"] = df_val["Ours"].replace("asian","Asian")
  df_val["Ours"] = df_val["Ours"].replace("latino","White")
  df_val["Ours"] = df_val["Ours"].replace("hispanic","White")
  df_val["Ours"] = df_val["Ours"].replace("black","Black")
  df_val["Ours"] = df_val["Ours"].replace("white","White")
  df_val["Ours"] = df_val["Ours"].replace("african","Black")
  df_val["Ours"] = df_val["Ours"].replace("asian","Asian")
  df_val["Ours"] = df_val["Ours"].replace("indian","Indian")

  acc = sum(df_val["Ours"] == df_val["label2"]) / len(df_val)
  model_acc[m_path.stem] = acc


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
morgan_1

{'ethni_081f1_v7': 0.7,
 'ethni_0825f1_v6': 0.86,
 'ethni_083f_unfrezee_V9': 0.84,
 'ethni_0857f1_v5': 0.82,
 'ethni_0858f_unfrezee_V9': 0.86,
 'ethni_085b_96unfrezee': 0.78,
 'ethni_085b_96unfrezee_V6': 0.84,
 'ethni_085b_unfrezee': 0.68,
 'ethni_085b_unfrezee_V6': 0.84,
 'ethni_086f1': 0.84,
 'ethni_088': 0.48,
 'ethni_088f1': 0.76,
 'ethni_089': 0.22,
 'ethni_08f1_v7': 0.7,
 'ethni_09': 0.48,
 'ethni_090': 0.48,
 'ethni_091': 0.48,
 'ethni_09136': 0.5,
 'ethni_09167': 0.56,
 'ethni_092': 0.6,
 'ethni_09217': 0.56,
 'ethni_0929b': 0.68,
 'ethni_092b': 0.78,
 'ethni_0933': 0.72,
 'ethni_new_v8_0.82': 0.8,
 'ethni_new_v8_082': 0.8,
 'ethni_new_v9_08': 0.78}

In [ ]:
%cd ../

/content


In [ ]:
learner = load_learner(models_path/"ethni_0825f1_v6.pkl")

In [ ]:
learner.predict("/content/dataset/Val/african/f4a0db679815478861fb54637ba1ba07e9-23-morgan-freeman.2x.h473.w710.jpg")

('african',
 tensor(0),
 tensor([9.9850e-01, 8.2509e-04, 3.7196e-04, 2.7198e-04, 3.4039e-05, 1.8740e-12,
         3.1152e-13, 4.8893e-13, 2.8926e-13, 3.3053e-13, 1.6542e-13, 2.6309e-13,
         5.9405e-13, 9.8431e-13, 1.8255e-13, 9.5037e-14, 8.8576e-13, 5.0107e-13,
         7.0514e-13, 2.7360e-13, 2.5486e-13, 1.4654e-13, 3.9549e-13, 1.4269e-13,
         3.1694e-13, 3.5812e-13, 1.1516e-13, 5.8486e-13, 3.8600e-13, 2.4901e-13,
         4.0515e-13, 3.2959e-13, 1.0037e-12, 1.7726e-13, 2.2781e-13, 2.6645e-13,
         8.0428e-14, 2.5057e-13, 2.6693e-13, 3.3582e-13, 1.1622e-13, 2.6980e-13,
         1.2830e-13, 2.8470e-13, 1.7586e-13, 4.9225e-13, 7.9886e-14, 1.2558e-13,
         3.5640e-13, 4.1257e-13, 1.0833e-13, 1.5090e-13, 5.5702e-13, 1.8019e-13,
         1.3560e-13, 1.3065e-13, 3.2682e-13, 1.4807e-13, 1.0914e-12, 2.2884e-13,
         2.4873e-13, 2.1076e-13, 4.9228e-13, 4.3229e-13, 7.1763e-14, 8.9648e-14,
         3.9819e-13, 4.5198e-13, 7.7863e-13, 5.3997e-13, 3.4032e-13, 4.1117e-13,
    

In [ ]:
!wget 'https://i.guim.co.uk/img/media/8eec87781f44af5b562ae66c4545f95c97afca4f/0_91_3700_2220/master/3700.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=a5374611a91163312dcf3c24401aeb9c'
# !wget https://i.pinimg.com/originals/b2/cb/e6/b2cbe60c73e15a9097e7a8d82e717861.jpg 



--2020-12-24 08:51:57--  https://i.guim.co.uk/img/media/8eec87781f44af5b562ae66c4545f95c97afca4f/0_91_3700_2220/master/3700.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=a5374611a91163312dcf3c24401aeb9c
Resolving i.guim.co.uk (i.guim.co.uk)... 151.101.1.111, 151.101.65.111, 151.101.129.111, ...
Connecting to i.guim.co.uk (i.guim.co.uk)|151.101.1.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 183409 (179K) [image/jpeg]
Saving to: ‘3700.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=a5374611a91163312dcf3c24401aeb9c’

3700.jpg?width=1200 100%[===================>] 179.11K  --.-KB/s    in 0.02s   

2020-12-24 08:51:57 (10.9 MB/s) - ‘3700.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=a5374611a91163312dcf3c24401aeb9c’ saved [183409/183409]



In [ ]:
!ls 

 b2cbe60c73e15a9097e7a8d82e717861.jpg
 b2cbe60c73e15a9097e7a8d82e717861.jpg.1
 dataset
 FairFace
 fair_face_models
 gdrive
'images?q=tbn:ANd9GcRtcoH8G8Zdi3-q2LZmQvys0XFDFw3e4dQjXw'
'images?q=tbn:ANd9GcRtcoH8G8Zdi3-q2LZmQvys0XFDFw3e4dQjXw.1'
'images?q=tbn:ANd9GcRtcoH8G8Zdi3-q2LZmQvys0XFDFw3e4dQjXw.2'
'images?q=tbn:ANd9GcRtcoH8G8Zdi3-q2LZmQvys0XFDFw3e4dQjXw&usqp=CAU'
 morgan2.jpg
 morgan.jpg
 sample_data


In [ ]:
from tqdm import tqdm_notebook as tqdm
val_path= base_dir/ "Val"

for img in tqdm(["morgan.jpg","morgan2.jpg"], desc="Images"):
  try: 
    im = Image.open(img) 
    for face in detector.detect_faces(np.asarray(im)):
      box = face["box"]
      im1 = im.crop(box=(box[0], box[1], box[0]+box[2],box[1]+box[3] )) 
      im1.save(img)
      break
  except IsADirectoryError:
    pass

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
learner.predict("morgan.jpg")

('african',
 tensor(0),
 tensor([9.9853e-01, 9.7169e-05, 7.1462e-04, 6.3777e-04, 1.9409e-05, 1.9322e-14,
         3.5104e-15, 5.6150e-15, 2.5385e-15, 2.8951e-15, 1.2112e-15, 2.2072e-15,
         6.2210e-15, 9.1783e-15, 1.2846e-15, 7.6048e-16, 6.4754e-15, 3.6239e-15,
         6.5987e-15, 2.6921e-15, 2.0084e-15, 9.5080e-16, 3.6611e-15, 8.2125e-16,
         3.1807e-15, 2.9849e-15, 9.1208e-16, 3.2640e-15, 3.4122e-15, 1.7819e-15,
         1.6253e-15, 3.7522e-15, 7.5584e-15, 1.5105e-15, 1.1293e-15, 1.7729e-15,
         5.2754e-16, 1.2442e-15, 2.7804e-15, 3.5950e-15, 7.3698e-16, 1.8355e-15,
         9.7314e-16, 3.5064e-15, 2.0876e-15, 3.4206e-15, 5.6981e-16, 1.2684e-15,
         6.0015e-15, 3.9095e-15, 6.9620e-16, 8.6990e-16, 3.5836e-15, 1.7039e-15,
         9.8014e-16, 1.0943e-15, 2.7142e-15, 1.3442e-15, 6.0373e-15, 1.8581e-15,
         1.8738e-15, 2.7037e-15, 5.7294e-15, 1.8127e-15, 5.8977e-16, 8.8063e-16,
         1.4311e-15, 3.4380e-15, 7.0476e-15, 3.1159e-15, 2.9258e-15, 3.1568e-15,
    

In [ ]:
learner.predict("morgan2.jpg")

('african',
 tensor(0),
 tensor([9.1793e-01, 4.1993e-02, 3.0006e-02, 1.0055e-02, 1.5306e-05, 1.9797e-12,
         1.7891e-13, 3.5560e-13, 1.7843e-13, 3.5238e-13, 1.4412e-13, 1.4453e-13,
         3.3555e-13, 2.1323e-13, 1.0637e-13, 3.7138e-14, 4.4031e-13, 2.9381e-13,
         3.5027e-13, 1.2256e-13, 1.1081e-13, 8.0765e-14, 2.4937e-13, 8.2678e-14,
         2.5243e-13, 1.6389e-13, 6.3038e-14, 1.8487e-13, 1.8283e-13, 1.0925e-13,
         1.6511e-13, 1.1658e-13, 8.7044e-13, 1.4471e-13, 1.0599e-13, 8.6533e-14,
         4.0329e-14, 1.4891e-13, 1.9634e-13, 1.4495e-13, 7.1498e-14, 9.6939e-14,
         7.1949e-14, 1.2311e-13, 6.2809e-14, 3.5629e-13, 3.3355e-14, 2.9967e-14,
         1.4594e-13, 2.2418e-13, 2.8150e-14, 4.4543e-14, 2.3584e-13, 9.4348e-14,
         6.0800e-14, 5.9429e-14, 2.1784e-13, 6.9961e-14, 4.5763e-13, 1.2207e-13,
         8.0530e-14, 1.0478e-13, 3.6870e-13, 2.2658e-13, 1.7224e-14, 3.7424e-14,
         2.7469e-13, 1.6001e-13, 3.3134e-13, 1.9677e-13, 2.5533e-13, 2.0372e-13,
    